
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Simulate Astronomical Tide using ttide library

inputs required: 
  * Astronomical Tide historical time series at the study site 

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op

# python libs
import numpy as np
import xarray as xr
from datetime import datetime, timedelta

# custom libs
from ttide.t_tide import t_tide
from ttide.t_predic import t_predic

# https://www.eoas.ubc.ca/~rich/#T_Tide

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.plotting.tides import Plot_AstronomicalTide, Plot_ValidateTTIDE



## Database and Site parameters

In [ ]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


In [ ]:
# --------------------------------------
# Load astronomical tide historical and set simulation dates

_, TIDE_astro = db.Load_TIDE_hist()          # astronomical tide historical data
TIDE = TIDE_astro.predicted

# Simulation dates
d1_sim = np.datetime64('1700-01-01')
d2_sim = np.datetime64('2700-01-01')



## Astronomical Tide

In [ ]:
# --------------------------------------
# astronomical tide data

# remove tide nanmin
TIDE = TIDE - np.nanmin(TIDE)

# data length has to be lesser than 18.6 years
dt_cut = np.datetime64('1998-06-01')
TIDE = TIDE.where(TIDE.time >= dt_cut, drop=True)

# Plot astronomical tide
time = TIDE.time.values[:]
tide = TIDE.values[:]

Plot_AstronomicalTide(time, tide);



## Astronomical Tide - Fitting

In [ ]:
# --------------------------------------
# t_tide library - Fit

# TODO: ttide ha de se calibrada con un year de datos (no 18.6)
lat0 = 9.75
d_out = t_tide(TIDE.values, dt=1, lat=np.array(lat0))

# variables used for prediction
names = d_out['nameu']
freq = d_out['fu']
tidecon = d_out['tidecon']


In [ ]:
# --------------------------------------
# t_tide library - Validation

d1_val = TIDE.time.values[0]
d2_val = TIDE.time.values[-1]
date_val = np.arange(d1_val, d2_val, dtype='datetime64[h]')

tide_tt = t_predic(
    date_val, names, freq, tidecon,
    lat=lat0, ltype='nodal')

# Plot ttide validation 
time = TIDE.time.values[:-1]
tide = TIDE.values[:-1]
tide = tide - np.nanmean(tide)

Plot_ValidateTTIDE(time, tide, tide_tt);



## Astronomical Tide - Prediction

In [ ]:
# --------------------------------------
# t_tide library - Prediction

date_pred = np.arange(d1_sim, d2_sim + np.timedelta64(1,'h'), dtype='datetime64[h]')
atide_pred = t_predic(
    date_pred, names, freq, tidecon,
    lat=lat0, ltype='nodal')

# store data
ASTRO_sim = xr.Dataset(
    {
        'astro'   :(('time',), atide_pred),
    },
    {'time' : date_pred.astype(datetime)}
)

print(ASTRO_sim)

db.Save_TIDE_sim_astro(ASTRO_sim)


In [ ]:
# Plot astronomical tide prediction
time = ASTRO_sim.time.values[:]
tide = ASTRO_sim.astro.values[:]

Plot_AstronomicalTide(time, tide);
